# 准备数据集

In [9]:
import spacy
import torch
from datasets import load_dataset

# Training hyperparameters
TOTAL_DATA_NUM = 2000

MIN_FREQ = 1
batch_size = 64
epochs = 500
learning_rate = 3e-4
batch_first = True
pad_idx = 0
load_model = True
save_model = True

# Model hyperparameters
embedding_size = 256
num_heads = 4
dropout = 0.1
num_encoder_layers = 2
num_decoder_layers = 2
dim_feedforward = 1024
max_len = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device {device}')


# load tokenize model
spacy_en = spacy.load("en_core_web_sm")
spacy_zh = spacy.load("zh_core_web_sm")

# tokenize
def tokenize_en(text):
    return [tok.text.lower() for tok in spacy_en(text)]
def tokenize_zh(text):
    return [tok.text.lower() for tok in spacy_zh(text)]

# dataset zh-en
raw_datasets = load_dataset("wmt19", "zh-en")
print(raw_datasets) # 25984574 3981

train_dataset = raw_datasets["train"].select(range(TOTAL_DATA_NUM))
vali_dataset = raw_datasets["validation"]
print(train_dataset)
print(vali_dataset)

# tokenize dataset
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset

class WordDataset(Dataset):
    def __init__(self, dataset):
        self.tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')
        self.tokenizer_zh = get_tokenizer('spacy', language='zh_core_web_sm')
        self.en_lines = []
        self.zh_lines = []
        for data in dataset:
            self.en_lines.append(data['translation']['en'])
            self.zh_lines.append(data['translation']['zh'])

    def __len__(self):
        return len(self.en_lines)
    
    def __getitem__(self, idx):
        en_line = self.en_lines[idx]
        zh_line = self.zh_lines[idx]
        en_tokens = self.tokenizer_en(en_line)
        zh_tokens = self.tokenizer_zh(zh_line)
        return en_tokens, zh_tokens

Using device cpu


Using the latest cached version of the dataset since wmt19 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'zh-en' at C:\Users\21198\.cache\huggingface\datasets\wmt19\zh-en\0.0.0\c8a87a86736e7bf10c74c22e9033eba9146774e9 (last modified on Wed Jul  9 13:39:27 2025).


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 25984574
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3981
    })
})
Dataset({
    features: ['translation'],
    num_rows: 2000
})
Dataset({
    features: ['translation'],
    num_rows: 3981
})


In [10]:
# build vocab (opt)
from torch.utils.data import ConcatDataset
from collections import Counter
import pickle
import itertools


en_vocab = {
    '<pad>':0,
    '<sos>':1,
    '<eos>':2,
    '<unk>':3
}
zh_vocab = {
    '<pad>':0,
    '<sos>':1,
    '<eos>':2,
    '<unk>':3
}

tk_dataset = WordDataset(train_dataset)
en_all_tokens = list(itertools.chain.from_iterable(en_tokens for en_tokens, zh_tokens in tk_dataset))
zh_all_tokens = list(itertools.chain.from_iterable(zh_tokens for en_tokens, zh_tokens in tk_dataset))
en_vocab_counter = Counter(en_all_tokens)
zh_vocab_counter = Counter(zh_all_tokens)
# print(len(en_all_tokens))
# print(len(zh_all_tokens))


en_tot = 4
for index, (token, freq) in enumerate(en_vocab_counter.items()):
    if freq >= MIN_FREQ:
        en_vocab.update({token: en_tot})
        en_tot += 1  
zh_tot = 4
for index, (token, freq) in enumerate(zh_vocab_counter.items()):
    if freq >= MIN_FREQ:
        zh_vocab.update({token: zh_tot})
        zh_tot += 1    
print(len(en_vocab))
print(len(zh_vocab))


with open('vocab.en', "wb") as en_vocab_file:
    pickle.dump(en_vocab, en_vocab_file)
with open('vocab.zh', "wb") as zh_vocab_file: # write binary
    pickle.dump(zh_vocab, zh_vocab_file)

7324
6916


In [11]:
# define dataloader for model forward train
from torch.utils.data import DataLoader
import torch
from torch.nn.utils.rnn import pad_sequence

# word to number with vocab -- collate_fn
def collate_fn(batch):
    """批处理函数，处理变长序列"""
    zh_batch, en_batch = [], []
    
    for en_tokens, zh_tokens in batch:
        
        en_itokens = [en_vocab.get(token, en_vocab['<unk>']) for token in en_tokens] + [en_vocab.get('<eos>', 2)]
        zh_itokens = [zh_vocab.get(token, zh_vocab['<unk>']) for token in zh_tokens] + [zh_vocab.get('<eos>', 2)]
        
        en_batch.append(torch.tensor(en_itokens, dtype=torch.long))
        zh_batch.append(torch.tensor(zh_itokens, dtype=torch.long))
    
    en_batch = pad_sequence(en_batch, padding_value=en_vocab['<pad>'], batch_first=batch_first) # 用0填充短序列
    zh_batch = pad_sequence(zh_batch, padding_value=zh_vocab['<pad>'], batch_first=batch_first)
    
    return  en_batch, zh_batch

train_dataloader = DataLoader(
    WordDataset(train_dataset),
    batch_size=batch_size,
    shuffle=True,
    collate_fn = collate_fn,
    
)
vali_dataloader = DataLoader(
    WordDataset(vali_dataset),
    batch_size=batch_size,
    shuffle=True,
    collate_fn = collate_fn,
)

# 准备模型类

In [12]:
# define model
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(
            self,
            embedding_size,
            src_vocab_size,
            trg_vocab_size,
            num_heads,
            dropout,
            num_encoder_layers,
            num_decoder_layers,
            pad_idx,
            dim_feedforward,
            max_len,
            device
    ):
        super().__init__()

        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            dim_feedforward,
            dropout,
            batch_first=batch_first,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.pad_idx = pad_idx
    
    def make_padding_mask(self, N_S): # True will be ignored
        padding_mask = N_S == self.pad_idx
        return padding_mask.to(self.device)
    
    def forward(self, src, trg):
        batch_size, src_seq_length = src.shape
        batch_size, trg_seq_length = trg.shape

        src_position = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, batch_size)
            .transpose(0,1)
            .to(self.device)
        )
        trg_position = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, batch_size)
            .transpose(0,1)
            .to(self.device)
        )
        
        src_embed = self.dropout(
            self.src_word_embedding(src) + self.src_position_embedding(src_position)
        ) 
        trg_embed = self.dropout(
            self.trg_word_embedding(trg) + self.trg_position_embedding(trg_position)
        )

        src_padding_mask = self.make_padding_mask(src)
        trg_padding_mask = self.make_padding_mask(trg)
        trg_mask = (
            torch.triu(
                torch.ones(trg_seq_length, trg_seq_length, dtype=torch.bool, device=self.device),
                diagonal=1
            )
        )

        out = self.transformer(
            src_embed,
            trg_embed,
            src_key_padding_mask = src_padding_mask,
            tgt_key_padding_mask = trg_padding_mask,
            tgt_mask = trg_mask,
        )
        out = self.fc_out(out)
        return out

In [13]:
# 翻译测试函数
def translate_sentence(model, sentence, en_vocab, zh_vocab, device, max_length=50):
    model.eval()
    
    # 分词并转换为索引
    tokens = [en_vocab.get(token, en_vocab['<unk>']) for token in tokenize_en(sentence)] + [en_vocab.get('<eos>', 2)]
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device)  # [1, seq_len]
    
    # 初始化目标序列
    outputs = [zh_vocab.get('<sos>', 1)]
    
    with torch.no_grad():
        for i in range(max_length):
            trg_tensor = torch.LongTensor(outputs).unsqueeze(0).to(device)  # [1, seq_len]
            output = model(src_tensor, trg_tensor)
            best_guess = output.argmax(2)[:,-1].item()
            outputs.append(best_guess)
            
            if best_guess == zh_vocab.get('<eos>', 2):
                print("FIND EOS")
                break
    
    # 转换索引为词汇
    zh_vocab_reverse = {v: k for k, v in zh_vocab.items()}
    translated_sentence = [zh_vocab_reverse.get(idx, '<unk>') for idx in outputs[1:]]
    return ''.join(translated_sentence)

In [14]:
# 转换函数
def translate_to_sentence(tokens, vocab, is_chinese=False):
    vocab_reverse = {v: k for k, v in vocab.items()}

    if hasattr(tokens, 'cpu'):
        token_list = tokens.cpu().numpy().tolist()
    else:
        token_list = tokens.tolist()
    
    # 转换为词汇
    words = []
    for token_id in token_list:
        word = vocab_reverse.get(token_id, '<unk>')
        words.append(word)
        if word == '<eos>':  # 遇到结束符就停止
            break
    
    # 根据语言类型连接
    if is_chinese:
        return ''.join(words)  # 中文不需要空格
    else:
        return ' '.join(words)  # 英文用空格连接

# 训练模型

In [15]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [16]:
# prepared dataloader and model! start train
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

src_vocab_size = len(en_vocab)
trg_vocab_size = len(zh_vocab)

# record loss with tensorboard
writer = SummaryWriter("runs/loss_plot")
step = 0

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    num_heads,
    dropout,
    num_encoder_layers,
    num_decoder_layers,
    pad_idx,
    dim_feedforward,
    max_len,
    device,
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, 
)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)

for epoch in range(epochs):
    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)
    
    # 获取验证集样本用于翻译测试
    test_sample = train_dataset[step]['translation']
    test_en_sentence = test_sample['en']
    test_zh_reference = test_sample['zh']
    print(f"测试句子 (英文): {test_en_sentence}")
    # 每个epoch进行翻译测试
    translated = translate_sentence(model, test_en_sentence, en_vocab, zh_vocab, device)
    print(f"翻译结果: {translated}")
    print(f"参考翻译: {test_zh_reference}\n")

    model.train()
    losses = []

    progress_bar = tqdm(enumerate(train_dataloader), 
                       total=len(train_dataloader),
                       desc=f"Epoch {epoch+1}",
                       ncols=100)
    
    for _, (en_batch, zh_batch) in progress_bar:
        input = en_batch.to(device)
        target = zh_batch.to(device)
        # for i, (en_tokens, zh_tokens) in enumerate(zip(input, target)):
        #     print(f'[pair{i+1}]')
        #     print(f'English: {translate_to_sentence(en_tokens, en_vocab, is_chinese=False)}')
        #     print(f'Chinese: {translate_to_sentence(zh_tokens, zh_vocab, is_chinese=True)}')
        #     print()  # 添加空行便于阅读

        #     # 只打印前几个样本避免输出过多
        #     if i >= 3:
        #         break
            

        right_shift = torch.ones(target.shape[0], 1, dtype=target.dtype, device=target.device)
        shifted_target = torch.cat([right_shift, target], dim=1)
        
        output = model(input, shifted_target)

        output = output[:,:-1,:].reshape(-1, output.shape[2])
        target = target.reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)
        losses.append(loss.item())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=3)
        optimizer.step()

        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1

        progress_bar.set_postfix({'Loss': f'{loss.item():.4f}'})
    
    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)

C:\Users\21198\AppData\Local\Temp\ipykernel_28136\1781684520.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  load_checkpoint(torch.load("my_checkpoint.pth.tar"), model,

=> Loading checkpoint
=> Saving checkpoint
测试句子 (英文): 1929 or 1989?


c:\Users\21198\anaconda3\envs\test\lib\site-packages\torch\nn\modules\transformer.py:502: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\b\abs_dd5ga5p9x7\croot\libtorch_1746637520098\work\aten\src\ATen\NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(


FIND EOS
翻译结果: 的的的的的的的的的的的的的的的的的的的的的的的。<eos>
参考翻译: 1929年还是1989年?



Epoch 1: 100%|█████████████████████████████████████████| 32/32 [00:53<00:00,  1.67s/it, Loss=6.6952]


...

=> Saving checkpoint
测试句子 (英文): European policymakers today often complain that, were it not for the US financial crisis, the eurozone would be doing just fine.
FIND EOS
翻译结果: 但是，欧洲决策者不会消失，就像金融危机，但金融危机，人们必须解决方案，而不是危机，那么我们必须金融危机前金融危机都不会引发金融危机并专注于。<eos>
参考翻译: 如今，欧洲决策者经常抱怨说，要不是美国金融危机，欧元区根本不会遭殃。



Epoch 41: 100%|████████████████████████████████████████| 32/32 [01:19<00:00,  2.50s/it, Loss=1.4194]


=> Saving checkpoint
测试句子 (英文): That doesn’t mean that we are advocating a move to socialism; we want better and smarter regulation, not regulation for its own sake.
FIND EOS
翻译结果: 我们并不意味着我们需要我们需要更多的更多地；我们无法获得和监管而监管。<eos>
参考翻译: 这并不意味着我们鼓吹社会主义；监管必须更好和更明智，而不是为监管而监管。



Epoch 42:  12%|█████▏                                   | 4/32 [00:23<02:41,  5.77s/it, Loss=1.2980]


KeyboardInterrupt: 